In [ ]:
# Import libraries and dependencies

import numpy as np
import pandas as pd
import time
from finsymbols import symbols

import api_calls

In [ ]:
sp500 = symbols.get_sp500_symbols()
listings = api_calls.call_listings()

# Create a DataFrame from the listings data, with the first row as the column names
all_stocks = pd.DataFrame(listings[1:], columns=listings[0])

In [ ]:
# Get the symbols from the S&P 500
sp500_symbols = [sp500[i]['symbol'] for i in range(len(sp500))]

# Remove \n from the symbols
sp500_symbols = [sp500_symbols[i].replace('\n', '') for i in range(len(sp500_symbols))]

# Replace "BF.B" with "BF-B"
sp500_symbols = [sp500_symbols[i].replace('BF.B', 'BF-B') for i in range(len(sp500_symbols))]

# Display sample data
sp500_symbols

In [ ]:
# Remove all ETFs from the DataFrame
all_stocks = all_stocks[all_stocks['assetType'] != 'ETF']

# Remove all stocks that are not on the NYSE or NASDAQ
all_stocks = all_stocks[(all_stocks['exchange'] == 'NYSE') | (all_stocks['exchange'] == 'NASDAQ')]

# Create a list with the symbols from all stocks
all_stocks_symbols = all_stocks['symbol'].tolist()

all_stocks_symbols

In [ ]:
# Create a function to calculate the 100-day return for stocks in the list
def get_returns(symbols):
    # Create an empty DataFrame with symbol and return columns
    df = pd.DataFrame(columns=['symbol', 'stock_return_100', 'market_return_100'])

    # Find the 100-day return for the market
    market_return_100 = get_return('VTI')

    for i, symbol in enumerate(symbols):
        print(symbol)
        # Calculate the 100-day return
        stock_return_100 = get_return(symbol)

        # Add the symbol, stock return, and market return to the DataFrame using pd.concat
        df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'stock_return_100': stock_return_100, 'market_return_100': market_return_100}, index=[0])], axis=0)

        # If the index is at 1200, pause for 60 seconds to avoid the API limit
        if i % 1150 == 0:
            time.sleep(60)

    return df

# Create a function to calculate the return for a stock over the last 100 days
def get_return(symbol):
    # Call the alpha vantage api to get the data
    stock_data = api_calls.call_daily(symbol)

    # Return None if there is no data or less than 100 days of data
    try:
        if len(stock_data['Time Series (Daily)']) < 100:
            return None
    except:
        return None

    # Get the adjusted closing prices
    stock_data_df = pd.DataFrame(stock_data['Time Series (Daily)']).T

    # Calculate the return from the first day to the last day
    return_100 = (float(stock_data_df['5. adjusted close'][0]) - float(stock_data_df['5. adjusted close'][-1])) / float(stock_data_df['5. adjusted close'][-1])

    return return_100

In [ ]:
# Create an empty DataFrame with symbol and return columns
df = pd.DataFrame(columns=['symbol', 'stock_return_100', 'market_return_100'])

# Indicate which symbols to use
symbols = all_stocks_symbols

In [ ]:
# Loop through the symbols and get the company overview data, calculating the 100-day return for each company and saving it to a DataFrame

df = get_returns(symbols)

In [ ]:
df

In [ ]:
print(api_calls.call_daily('AESI'))

In [ ]:
# Print the companies that have None for the 100-day return
df[df['stock_return_100'].isnull()]

In [ ]:
#Rerun the 100-day return data for the companies that have None for the 100-day return using get_returns
for symbol in df[df['stock_return_100'].isnull()]['symbol']:
    # Calculate the 100-day return
    stock_return_100 = get_return(symbol)

    # Replace the None value with the calculated return
    df.loc[df['symbol'] == symbol, 'stock_return_100'] = stock_return_100

    # Print the progress
    print(symbol, stock_return_100)

In [ ]:
df.to_pickle('progress.csv')

In [ ]:
df = pd.read_pickle('progress.csv')

In [ ]:
# Create a function to calculate the news sentiment for stocks in the list
def get_sentiment(symbols):
    # Create an empty DataFrame with symbol and sentiment columns
    df = pd.DataFrame(columns=['symbol', 'sentiment'])

    for i, symbol in enumerate(symbols):
        print(symbol)
        # Call the alpha vantage api to get the news data
        news = api_calls.call_news(symbol)

        # If there is no news data, return None
        try:
            if news['items'] == '0':
                df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'sentiment': None}, index=[0])], axis=0)
                continue
        except:
            df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'sentiment': None}, index=[0])], axis=0)
            continue

        # Create a Pandas DataFrame
        stock_news = pd.DataFrame(news['feed'])

        # Calculate the average sentiment
        sentiment = stock_news['overall_sentiment_score'].mean()

        # Add the symbol and sentiment to the DataFrame using pd.concat
        df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'sentiment': sentiment}, index=[0])], axis=0)

        # If the index is at 1200, pause for 60 seconds to avoid the API limit
        if i % 1150 == 0:
            time.sleep(60)

    return df

In [ ]:
# Append the news sentiment data to the company overview DataFrame
df = pd.merge(df, get_sentiment(symbols), on='symbol', how='left')

In [ ]:
df

In [ ]:
# Print the companies that have None for the sentiment
df[df['sentiment'].isnull()]

In [ ]:
# Rerun the sentiment data for the companies that have None for the sentiment
for symbol in df[df['sentiment'].isnull()]['symbol']:
    # Call the alpha vantage api to get the news data
    news = api_calls.call_news(symbol)

    # If there is no news data, return None
    try:
        if news['items'] == '0':
            df.loc[df['symbol'] == symbol, 'sentiment'] = None
            continue
    except:
        df.loc[df['symbol'] == symbol, 'sentiment'] = None
        continue

    # Create a Pandas DataFrame
    stock_news = pd.DataFrame(news['feed'])

    # Calculate the average sentiment
    sentiment = stock_news['overall_sentiment_score'].mean()

    # Replace the None with the sentiment
    df.loc[df['symbol'] == symbol, 'sentiment'] = sentiment

In [ ]:
df.to_pickle('progress.csv')

In [ ]:
df = pd.read_pickle('progress.csv')

In [ ]:
# Create a function to calculate the company overview data for stocks in the list
def get_overview(symbols):
    # Create an empty DataFrame with symbol and return columns
    df = pd.DataFrame(columns=['symbol', 'market_cap', 'ebitda', 'pe_ratio', 'peg_ratio', 'book_value', 'dividend_per_share', 'dividend_yield', 'eps', 'revenue_per_share', 'profit_margin', 'operating_margin', 'return_on_assets', 'return_on_equity', 'revenue', 'gross_profit', 'diluted_eps', 'quarterly_earnings_growth', 'quarterly_revenue_growth', 'trailing_pe', 'forward_pe', 'price_to_sales_ratio', 'price_to_book_ratio', 'ev_to_revenue', 'ev_to_ebitda', 'beta'])

    for i, symbol in enumerate(symbols):
        print(symbol)

        # Call the alpha vantage api to get the company overview data
        overview = api_calls.call_company_overview(symbol)

        # If there is no overview data, return None
        try:
            if overview['Symbol'] == '0':
                df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'market_cap': None, 'ebitda': None, 'pe_ratio': None, 'peg_ratio': None, 'book_value': None, 'dividend_per_share': None, 'dividend_yield': None, 'eps': None, 'revenue_per_share': None, 'profit_margin': None, 'operating_margin': None, 'return_on_assets': None, 'return_on_equity': None, 'revenue': None, 'gross_profit': None, 'diluted_eps': None, 'quarterly_earnings_growth': None, 'quarterly_revenue_growth': None, 'trailing_pe': None, 'forward_pe': None, 'price_to_sales_ratio': None, 'price_to_book_ratio': None, 'ev_to_revenue': None, 'ev_to_ebitda': None, 'beta': None}, index=[0])], axis=0)
                continue
        except:
            df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'market_cap': None, 'ebitda': None, 'pe_ratio': None, 'peg_ratio': None, 'book_value': None, 'dividend_per_share': None, 'dividend_yield': None, 'eps': None, 'revenue_per_share': None, 'profit_margin': None, 'operating_margin': None, 'return_on_assets': None, 'return_on_equity': None, 'revenue': None, 'gross_profit': None, 'diluted_eps': None, 'quarterly_earnings_growth': None, 'quarterly_revenue_growth': None, 'trailing_pe': None, 'forward_pe': None, 'price_to_sales_ratio': None, 'price_to_book_ratio': None, 'ev_to_revenue': None, 'ev_to_ebitda': None, 'beta': None}, index=[0])], axis=0)
            continue

        # Add the symbol and overview data to the DataFrame using pd.concat
        df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'market_cap': overview['MarketCapitalization'], 'ebitda': overview['EBITDA'], 'pe_ratio': overview['PERatio'], 'peg_ratio': overview['PEGRatio'], 'book_value': overview['BookValue'], 'dividend_per_share': overview['DividendPerShare'], 'dividend_yield': overview['DividendYield'], 'eps': overview['EPS'], 'revenue_per_share': overview['RevenuePerShareTTM'], 'profit_margin': overview['ProfitMargin'], 'operating_margin': overview['OperatingMarginTTM'], 'return_on_assets': overview['ReturnOnAssetsTTM'], 'return_on_equity': overview['ReturnOnEquityTTM'], 'revenue': overview['RevenueTTM'], 'gross_profit': overview['GrossProfitTTM'], 'diluted_eps': overview['DilutedEPSTTM'], 'quarterly_earnings_growth': overview['QuarterlyEarningsGrowthYOY'], 'quarterly_revenue_growth': overview['QuarterlyRevenueGrowthYOY'], 'trailing_pe': overview['TrailingPE'], 'forward_pe': overview['ForwardPE'], 'price_to_sales_ratio': overview['PriceToSalesRatioTTM'], 'price_to_book_ratio': overview['PriceToBookRatio'], 'ev_to_revenue': overview['EVToRevenue'], 'ev_to_ebitda': overview['EVToEBITDA'], 'beta': overview['Beta']}, index=[0])], axis=0)

        # If the index is at 1200, pause for 60 seconds to avoid the API limit
        if i % 1150 == 0:
            time.sleep(60)

    return df

In [ ]:
# Append the company overview data to the DataFrame
df = pd.merge(df, get_overview(symbols), on='symbol', how='left')

In [ ]:
df

In [ ]:
# Print the companies that have None for the beta
df[df['market_cap'].isnull()]

In [ ]:
# Rerun the api call for the companies that have None for the market cap
for symbol in df[df['beta'].isnull()]['symbol']:
    # Call the alpha vantage api to get the company overview data
    company_overview = api_calls.call_company_overview(symbol)

    # If there is no company overview data, return None
    try:
        if company_overview['MarketCapitalization'] == 'N/A':
            df.loc[df['symbol'] == symbol, 'market_cap'] = None
            continue
    except:
        df.loc[df['symbol'] == symbol, 'market_cap'] = None
        continue

    # Add the market cap, ebitda, pe ratio, peg ratio, book value, dividend per share, dividend yield, eps, revenue per share, profit margin, operating margin, return on assets, return on equity, revenue, gross profit, diluted eps, quarterly earnings growth, quarterly revenue growth, trailing pe, forward pe, price to sales ratio, price to book ratio, ev to revenue, ev to ebitda, beta to the DataFrame
    df.loc[df['symbol'] == symbol, 'market_cap'] = company_overview['MarketCapitalization']
    df.loc[df['symbol'] == symbol, 'ebitda'] = company_overview['EBITDA']
    df.loc[df['symbol'] == symbol, 'pe_ratio'] = company_overview['PERatio']
    df.loc[df['symbol'] == symbol, 'peg_ratio'] = company_overview['PEGRatio']
    df.loc[df['symbol'] == symbol, 'book_value'] = company_overview['BookValue']
    df.loc[df['symbol'] == symbol, 'dividend_per_share'] = company_overview['DividendPerShare']
    df.loc[df['symbol'] == symbol, 'dividend_yield'] = company_overview['DividendYield']
    df.loc[df['symbol'] == symbol, 'eps'] = company_overview['EPS']
    df.loc[df['symbol'] == symbol, 'revenue_per_share'] = company_overview['RevenuePerShareTTM']
    df.loc[df['symbol'] == symbol, 'profit_margin'] = company_overview['ProfitMargin']
    df.loc[df['symbol'] == symbol, 'operating_margin'] = company_overview['OperatingMarginTTM']
    df.loc[df['symbol'] == symbol, 'return_on_assets'] = company_overview['ReturnOnAssetsTTM']
    df.loc[df['symbol'] == symbol, 'return_on_equity'] = company_overview['ReturnOnEquityTTM']
    df.loc[df['symbol'] == symbol, 'revenue'] = company_overview['RevenueTTM']
    df.loc[df['symbol'] == symbol, 'gross_profit'] = company_overview['GrossProfitTTM']
    df.loc[df['symbol'] == symbol, 'diluted_eps'] = company_overview['DilutedEPSTTM']
    df.loc[df['symbol'] == symbol, 'quarterly_earnings_growth'] = company_overview['QuarterlyEarningsGrowthYOY']
    df.loc[df['symbol'] == symbol, 'quarterly_revenue_growth'] = company_overview['QuarterlyRevenueGrowthYOY']
    df.loc[df['symbol'] == symbol, 'trailing_pe'] = company_overview['TrailingPE']
    df.loc[df['symbol'] == symbol, 'forward_pe'] = company_overview['ForwardPE']
    df.loc[df['symbol'] == symbol, 'price_to_sales_ratio'] = company_overview['PriceToSalesRatioTTM']
    df.loc[df['symbol'] == symbol, 'price_to_book_ratio'] = company_overview['PriceToBookRatio']
    df.loc[df['symbol'] == symbol, 'ev_to_revenue'] = company_overview['EVToRevenue']
    df.loc[df['symbol'] == symbol, 'ev_to_ebitda'] = company_overview['EVToEBITDA']
    df.loc[df['symbol'] == symbol, 'beta'] = company_overview['Beta']

In [ ]:
df.to_pickle('progress.csv')

In [ ]:
df = pd.read_pickle('progress.csv')

In [ ]:
df.to_csv('stocks.csv')